<a href="https://colab.research.google.com/github/vasiliyeskin/Geo-FNO/blob/veskin/airfoils/naca_geofno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Geo-FNO](https://github.com/zongyi-li/Geo-FNO) test on colab


In [ ]:
!gdown https://drive.google.com/drive/folders/1YBuaoTdOSr_qzaow-G-iwvbUI7fiUzu8?usp=sharing -O /tmp/folder --folder

In [ ]:
!git clone https://github.com/vasiliyeskin/Geo-FNO.git

In [8]:
%cd Geo-FNO/

/content/Geo-FNO


In [9]:
!ls

Adam.py   elasticity   LICENSE	plasticity  utilities3.py
airfoils  FluidSolver  pipe	readme.md


In [10]:
import torch.nn.functional as F
import matplotlib.pyplot as plt
from timeit import default_timer
from utilities3 import *
from Adam import Adam

torch.manual_seed(0)
np.random.seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True